In [1]:
import SimpleITK as sitk
import os
import dosma as dm
import numpy as np
from dosma import preferences
from dosma.scan_sequences import QDess, CubeQuant, Cones
from dosma.tissues import FemoralCartilage
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

220607-12:51:36,863 nipype.utils WARNING:
	 A newer version (1.8.1) of nipy/nipype is available. You are using 1.7.0


In [2]:
fc = FemoralCartilage()

# Perform analyticial voxelwise T2 measurement for femoral cartilage
# while suppressing fat and fluid regions for higher SNR.
qdess = QDess.from_dicom("../lib/resource/data/qdess", verbose=True)

t2map = qdess.generate_t2_map(fc, suppress_fat=True, suppress_fluid=True)

# Clip the estimated T2 values between [0, 80]
t2map.volumetric_map = np.clip(t2map.volumetric_map, 0, 80)

  0%|          | 0/160 [00:00<?, ?it/s]

/Users/huozimu/opt/anaconda3/lib/python3.9/site-packages/dosma/scan_sequences/mri/qdess.py:217: RuntimeWarning: divide by zero encountered in true_divide
  ratio = mask * echo_2 / echo_1
/Users/huozimu/opt/anaconda3/lib/python3.9/site-packages/dosma/scan_sequences/mri/qdess.py:217: RuntimeWarning: invalid value encountered in true_divide
  ratio = mask * echo_2 / echo_1
/Users/huozimu/opt/anaconda3/lib/python3.9/site-packages/dosma/scan_sequences/mri/qdess.py:221: RuntimeWarning: overflow encountered in true_divide
  t2map = -2000 * (TR - TE) / (xp.log(abs(ratio) / k) + c1)
/Users/huozimu/opt/anaconda3/lib/python3.9/site-packages/dosma/scan_sequences/mri/qdess.py:221: RuntimeWarning: divide by zero encountered in log
  t2map = -2000 * (TR - TE) / (xp.log(abs(ratio) / k) + c1)


In [3]:
cq = CubeQuant.from_dicom("../lib/resource/data/cubequant", verbose=True, num_workers=4)

  0%|          | 0/288 [00:00<?, ?it/s]

In [4]:
def elastic(target, moving):
    target = target.astype(np.float64)
    moving = moving.astype(np.float64)
    fixed_image = sitk.GetImageFromArray(target)
    moving_image = sitk.GetImageFromArray(moving)
    initial_transform = sitk.CenteredTransformInitializer(
        fixed_image,
        moving_image,
        sitk.Euler3DTransform(),
        sitk.CenteredTransformInitializerFilter.GEOMETRY,
    )
    moving_resampled = sitk.Resample(
        moving_image,
        fixed_image,
        initial_transform,
        sitk.sitkLinear,
        0.0,
        moving_image.GetPixelID(),
    )
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)

    registration_method.SetInterpolator(sitk.sitkLinear)

    # Optimizer settings.
    registration_method.SetOptimizerAsGradientDescent(
        learningRate=1.0,
        numberOfIterations=100,
        convergenceMinimumValue=1e-6,
        convergenceWindowSize=10,
    )
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    registration_method.SetInitialTransform(initial_transform, inPlace=False)

    final_transform = registration_method.Execute(
        sitk.Cast(fixed_image, sitk.sitkFloat32), sitk.Cast(moving_image, sitk.sitkFloat32)
    )
    moving_resampled = sitk.Resample(
        moving_image,
        fixed_image,
        final_transform,
        sitk.sitkLinear,
        0.0,
        moving_image.GetPixelID(),
    )
    targ = sitk.GetArrayViewFromImage(fixed_image)
    ref = sitk.GetArrayFromImage(moving_resampled)
    return targ.astype(np.float64), ref.astype(np.float64)

In [5]:
im1 = t2map.volumetric_map.A
im2 = cq.volumes[0].A

In [6]:
t1, t2 = elastic(im1, im2)

In [8]:
def show(ramp):
    _, ax = plt.subplots(1,1, figsize=(5,5))
    plt.imshow(im1[...,int(ramp)], cmap="gray")
    plt.imshow(im2[...,int(ramp)], cmap="inferno", alpha=0.5, interpolation="bilinear")

interact(show, ramp = widgets.FloatSlider(value=10,
                                               min=0,
                                               max=80,
                                               step=1))

interactive(children=(FloatSlider(value=10.0, description='ramp', max=80.0, step=1.0), Output()), _dom_classes…

<function __main__.show(ramp)>